In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from datetime import datetime,timedelta
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier
import datetime
import holidays
from sklearn.preprocessing import LabelEncoder
import prophet
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, plot_importance
from pre_processing_sy import *
# from sktime.forecasting.model_selection import temporal_train_test_split
# from sktime.utils.plotting import plot_series

# 데이터 불러오기

In [2]:
# 데이터 불러오기
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
trade = pd.read_csv("dataset/international_trade.csv")
submission = pd.read_csv("dataset/sample_submission.csv")

# 예측하기

In [3]:
# 일별 인덱스 추가해보기(train)
train_pre, test_pre = pre_all(train, test)
temp = pd.melt(pd.pivot(train_pre, columns = ["item","corporation","location"], index = "timestamp", values = "price").reset_index().reset_index(), id_vars = ["timestamp", "index"])
train_pre = pd.merge(train_pre, temp, how = "left", on = ["timestamp","item","corporation","location"]).drop(columns = ["supply","value","index"])

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 16)
전처리 후 test 크기 : (1092, 16)


In [4]:
# 일별 인덱스 추가해보기(test)
temp = pd.melt(pd.pivot(test_pre, columns = ["item","corporation","location"], index = "timestamp", values = "price").reset_index().reset_index(), id_vars = ["timestamp", "index"])
test_pre = pd.merge(test_pre, temp, how = "left", on = ["timestamp","item","corporation","location"]).drop(columns = ["supply","price","value","index"])
test_pre.head(5)

,ID,timestamp,item,corporation,location,year,month,day,week_day,year_month,week,week_num,season,holiday
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,5,50,9,219,1,0
1,TG_E_S_20230304,2023-03-04,TG,E,S,2023,3,4,5,50,9,219,1,0
2,BC_B_J_20230304,2023-03-04,BC,B,J,2023,3,4,5,50,9,219,1,0
3,TG_E_J_20230304,2023-03-04,TG,E,J,2023,3,4,5,50,9,219,1,0
4,BC_B_S_20230304,2023-03-04,BC,B,S,2023,3,4,5,50,9,219,1,0


In [5]:
train_tg= pd.get_dummies(train_pre, columns = ["item","corporation","location"]).drop(columns = ["ID", "season"])
train_tg.head(5)

,timestamp,price,year,month,day,week_day,year_month,week,week_num,holiday,...,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2019-01-01,0.0,2019,1,1,1,0,1,1,1,...,0,1,1,0,0,0,0,0,1,0
1,2019-01-01,0.0,2019,1,1,1,0,1,1,1,...,0,0,1,0,0,0,0,0,0,1
2,2019-01-01,0.0,2019,1,1,1,0,1,1,1,...,1,0,0,0,0,1,0,0,1,0
3,2019-01-01,0.0,2019,1,1,1,0,1,1,1,...,0,0,0,0,0,1,0,0,1,0
4,2019-01-01,0.0,2019,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,1,1,0


In [6]:
cut_off = "2023-02-18"
x_train_sup= train_tg[train_tg["timestamp"]<cut_off].drop(columns = ["timestamp","price"])
x_valid_sup = train_tg[train_tg["timestamp"]>=cut_off].drop(columns = ["timestamp","price"])
y_train_sup, y_valid_sup = train_tg[train_tg["timestamp"]<cut_off]["price"] ,train_tg[train_tg["timestamp"]>=cut_off]["price"]
x_train_sup.head(5)

,year,month,day,week_day,year_month,week,week_num,holiday,item_BC,item_CB,...,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2019,1,1,1,0,1,1,1,0,0,...,0,1,1,0,0,0,0,0,1,0
1,2019,1,1,1,0,1,1,1,0,1,...,0,0,1,0,0,0,0,0,0,1
2,2019,1,1,1,0,1,1,1,0,0,...,1,0,0,0,0,1,0,0,1,0
3,2019,1,1,1,0,1,1,1,1,0,...,0,0,0,0,0,1,0,0,1,0
4,2019,1,1,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,1,1,0


In [7]:
### early stopping 
evals = [(x_valid_sup, y_valid_sup)]

xgb_wrapper = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)
xgb_wrapper.fit(x_train_sup, y_train_sup, early_stopping_rounds =100, eval_set = evals, eval_metric = "rmse", verbose=True)
pred_sup = xgb_wrapper.predict(x_valid_sup)
for idx in range(len(pred_sup)):
    if pred_sup[idx]<0:
        pred_sup[idx]=0
print(np.sqrt(mean_squared_error(y_valid_sup, pred_sup)))

[0]	validation_0-rmse:2055.85738
[1]	validation_0-rmse:2036.86765
[2]	validation_0-rmse:2018.16589
[3]	validation_0-rmse:1999.60851
[4]	validation_0-rmse:1981.30947
[5]	validation_0-rmse:1963.20301
[6]	validation_0-rmse:1945.32574


C:\Users\sybae\anaconda3\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\sybae\anaconda3\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[7]	validation_0-rmse:1927.51621
[8]	validation_0-rmse:1910.06038
[9]	validation_0-rmse:1892.78611
[10]	validation_0-rmse:1875.63087
[11]	validation_0-rmse:1858.75180
[12]	validation_0-rmse:1842.12096
[13]	validation_0-rmse:1825.62593
[14]	validation_0-rmse:1809.37008
[15]	validation_0-rmse:1793.30801
[16]	validation_0-rmse:1777.47527
[17]	validation_0-rmse:1761.78502
[18]	validation_0-rmse:1746.32301
[19]	validation_0-rmse:1730.99268
[20]	validation_0-rmse:1715.93472
[21]	validation_0-rmse:1700.97175
[22]	validation_0-rmse:1686.27404
[23]	validation_0-rmse:1671.83867
[24]	validation_0-rmse:1657.44689
[25]	validation_0-rmse:1643.24064
[26]	validation_0-rmse:1629.16420
[27]	validation_0-rmse:1615.29851
[28]	validation_0-rmse:1601.61127
[29]	validation_0-rmse:1588.12681
[30]	validation_0-rmse:1574.83720
[31]	validation_0-rmse:1561.88197
[32]	validation_0-rmse:1549.04070
[33]	validation_0-rmse:1536.26290
[34]	validation_0-rmse:1523.72810
[35]	validation_0-rmse:1511.47299
[36]	validation_0

[249]	validation_0-rmse:722.36589
[250]	validation_0-rmse:722.15615
[251]	validation_0-rmse:722.20022
[252]	validation_0-rmse:722.29452
[253]	validation_0-rmse:721.96175
[254]	validation_0-rmse:722.09756
[255]	validation_0-rmse:722.14168
[256]	validation_0-rmse:721.89803
[257]	validation_0-rmse:722.02026
[258]	validation_0-rmse:722.34803
[259]	validation_0-rmse:722.11664
[260]	validation_0-rmse:722.43222
[261]	validation_0-rmse:722.74392
[262]	validation_0-rmse:722.99825
[263]	validation_0-rmse:723.37662
[264]	validation_0-rmse:723.61113
[265]	validation_0-rmse:724.17960
[266]	validation_0-rmse:724.62011
[267]	validation_0-rmse:725.05852
[268]	validation_0-rmse:725.45829
[269]	validation_0-rmse:725.92387
[270]	validation_0-rmse:726.41285
[271]	validation_0-rmse:726.73997
[272]	validation_0-rmse:727.36550
[273]	validation_0-rmse:728.01040
[274]	validation_0-rmse:728.60936
[275]	validation_0-rmse:729.14837
[276]	validation_0-rmse:729.07217
[277]	validation_0-rmse:729.65750
[278]	validati

In [8]:
### test 해보기
test_no_sort =  pd.get_dummies(test_pre, columns = ["item","corporation","location"]).drop(columns = ["ID", "season","timestamp"])
test_no_sort.head(5)

,year,month,day,week_day,year_month,week,week_num,holiday,item_BC,item_CB,...,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2023,3,4,5,50,9,219,0,0,0,...,0,1,1,0,0,0,0,0,1,0
1,2023,3,4,5,50,9,219,0,0,0,...,0,1,0,0,0,0,1,0,0,1
2,2023,3,4,5,50,9,219,0,1,0,...,0,0,0,1,0,0,0,0,1,0
3,2023,3,4,5,50,9,219,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,2023,3,4,5,50,9,219,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [9]:
print(f"test의 컬럼 : {test_no_sort.columns}")
print(f"train의 컬럼 : {x_valid_sup.columns}")
print(f"test의 컬럼 : {len(test_no_sort.columns)}")
print(f"train의 컬럼 : {len(x_valid_sup.columns)}")

test의 컬럼 : Index(['year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S'],
      dtype='object')
train의 컬럼 : Index(['year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S'],
      dtype='object')
test의 컬럼 : 21
train의 컬럼 : 21


In [10]:
### early stopping 
# evals = [(x_valid_sup, y_valid_sup)]

xgb_wrapper = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)
xgb_wrapper.fit(train_tg.drop(columns = ["timestamp","price"]), train_tg["price"], eval_metric = "rmse", verbose=True)
pred_test_nosort = xgb_wrapper.predict(test_no_sort)
for idx in range(len(pred_test_nosort)):
    if pred_test_nosort[idx]<0:
        pred_test_nosort[idx]=0

C:\Users\sybae\anaconda3\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


In [11]:
answer_no_sort = pd.concat([test_pre["ID"],pd.DataFrame(pred_test_nosort, columns = ["answer"])], axis = 1)
answer_no_sort.head(7)

,ID,answer
0,TG_A_J_20230304,3012.150635
1,TG_E_S_20230304,3406.471680
2,BC_B_J_20230304,2663.313232
3,TG_E_J_20230304,553.707703
4,BC_B_S_20230304,75.890663
5,TG_D_S_20230304,3589.088867
6,BC_C_J_20230304,2583.179688
